In [1]:
#Installing and Loading the required libraries 
!pip install -q -U transformers accelerate datasets scipy



import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.4.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires scipy<1.12,>=1.4.1, but you have scipy 1.13.1 which is incompatible.


In [2]:
#Loading the pretrained model
#pre-trained language model to be loaded from Hugging Face's model hub
llm_model = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"

#tokenizer corresponding to the model to convert text into a format that the model can understand
tokenizer = AutoTokenizer.from_pretrained(llm_model)



# Load the pre-trained causal language model with the specified quantization configuration
# `device_map="auto"`: Automatically assigns model layers to available GPUs, optimizing for performance
model = AutoModelForCausalLM.from_pretrained(
    llm_model, 
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

Mixtral 8x7B outperforms Llama 2 70B on most benchmarks with 6x faster inference. It is the strongest open-weight model with a permissive license and the best model overall regarding cost/performance trade-offs. In particular, it matches or outperforms GPT3.5 on most standard benchmarks.

Mixtral-8x7B is the second large language model (LLM) released by mistral.ai, after Mistral-7B.

In [3]:
# loading data from local directory in csv format
data = load_dataset('csv', data_files='/kaggle/input/newdata/New Auto Data.csv', encoding = 'ISO-8859-1')

# loading data from hugging face datasets
#data = load_dataset("harpreetsahota/modern-to-shakesperean-translation")
# extracting traning data
train_data = data['train']
# train_data view
train_data[2]

{'deposition': '` Beginning of Page 1 \nLine 1, 11:44:39: ll\nLine 2, 11:44:39: Gregory Vo\nLine 3, 11:44:39: a.\nLine 4, 11:44:39: a\nLine 5, 11:44:39: Plaintiff\nLine 6, 11:44:39: vs.\nLine 7, 11:44:39: ROOFING, INC., et\nLine 8, 11:44:39: Defendants\nLine 9, 11:44:39: The deposition\nLine 10, 11:44:39: on Rv |\nLine 11, 11:44:39: at the Law Offices of McNar\nLine 12, 11:44:39: Building, Suite 310, Two Ne\n End of Page 1 \n\n Beginning of Page 2 \nLine 1, 11:44:39: Baltimore, Maryland 21201-:\nLine 2, 11:44:39: Notary Public.\nLine 3, 11:44:39: APPEARANCES:\nLine 4, 11:44:39: LAURA G. ZOIS,\nLine 5, 11:44:39: On behalf o:\nLine 6, 11:44:39: J. GREGORY DON]\nLine 7, 11:44:39: On behalf o:\nLine 8, 11:44:39: ALSO PRESENT:\nLine 9, 11:44:39: REPORTED BY: Sandra A. Ju\nLine 10, 11:44:39: be\nLine 11, 11:44:39: Gore Brothers Reporting & Video Co., Inc.\nLine 12, 11:44:39: 410-837-3027\nLine 13, 11:44:39: Ao\n End of Page 2 \n\n Beginning of Page 3 \nLine 1, 11:44:39: slker - 6/5/02\nLine 

In [4]:

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_summary(query: str, model, tokenizer) -> str:
    # Define a simplified template for the prompt that will be given to the model
    prompt_template = """
    ## Instructions
    Summarize the deposition in approximately 50 words. Include the following:
    - Deponent’s name
    - Case details (name, number, court, date, location)
    - General topic of the case
    - Key topics discussed
    - Pivotal statements
    - Significant objections and rulings
    - Crucial discrepancies or admissions
    - Exhibits referenced
    
    ## Deposition
    {query}
    
    ## Summary
    """

    # Insert the actual deposition transcript into the prompt template
    prompt = prompt_template.format(query=query)
    # Tokenize the prompt, converting it to a format suitable for the model
    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    # Move the tokenized inputs to the appropriate device (GPU or CPU)
    model_inputs = encodeds.to(DEVICE)
    # Generate a response from the model
    generated_ids = model.generate(
        **model_inputs,                # Pass the model inputs
        max_new_tokens=500,            # Adjusted to limit the output to around 500 tokens
        do_sample=True,                # Enable sampling to generate diverse outputs
        pad_token_id=tokenizer.eos_token_id  # Use the end-of-sequence token for padding
    )
    # Decode the generated token IDs back into human-readable text
    decoded = tokenizer.batch_decode(generated_ids)
    generated_text = decoded[0]
    
    return generated_text


In [5]:
query = """
-- Beginning of Page 1 ---
Line 1, 10:32:18: In The
Line 2, 10:32:18: Matter Of:
Line 3, 10:32:18: Smith vs. City Construction Co.
--- End of Page 1 ---

--- Beginning of Page 2 ---
Line 1, 10:32:18:
--- End of Page 2 ---

--- Beginning of Page 3 ---
Line 1, 10:32:18: Court of Common Pleas
Line 2, 10:32:18: County of Newfield
Line 3, 10:32:18: Case No. 2023-CV-019284
--- End of Page 3 ---

--- Beginning of Page 4 ---
Line 1, 10:32:18: Deposition of Jane Doe,
Line 2, 10:32:18: held at: 123 Broad St., Newfield, NY
Line 3, 10:32:18: Before: Maria Gonzalez, Court Reporter
--- End of Page 4 ---

--- Beginning of Page 5 ---
Line 1, 10:32:18: APPEARANCES
Line 2, 10:32:18: COUNSEL FOR PLAINTIFF:
Line 3, 10:32:18: John Appleseed, Esq.
Line 4, 10:32:18: Appleseed & Associates
Line 5, 10:32:18: 456 Orchard Rd., Suite 300
Line 6, 10:32:18: Newfield, NY 10027
Line 7, 10:32:18: (555) 123-4567
--- End of Page 5 ---

--- Beginning of Page 6 ---
Line 1, 10:32:18: COUNSEL FOR DEFENDANT:
Line 2, 10:32:18: Sally Stone, Esq.
Line 3, 10:32:18: Stone Law Group
Line 4, 10:32:18: 789 Quarry Way, Suite 500
Line 5, 10:32:18: Newfield, NY 10031
Line 6, 10:32:18: (555) 987-6543
--- End of Page 6 ---

--- Beginning of Page 7 ---
Line 1, 10:32:18: Page 1
Line 2, 10:32:18: PROCEEDINGS
Line 3, 10:32:18: Whereupon,
Line 4, 10:32:18: Jane Doe, being first duly sworn, was examined and testified as follows:
Line 5, 10:32:18: DIRECT EXAMINATION
Line 6, 10:32:18: BY MR. APPLESEED:
--- End of Page 7 ---

--- Beginning of Page 8 ---
Line 1, 10:32:18: Q. Please state your name for the record.
Line 2, 10:32:18: A. My name is Jane Doe.
Line 3, 10:32:18: Q. Ms. Doe, how are you currently employed?
Line 4, 10:32:18: A. I am a project manager at City Construction Co.
--- End of Page 8 ---

--- Beginning of Page 9 ---
Line 1, 10:32:18: Q. Were you involved in the project at 500 Main Street on January 5th, 2023?
Line 2, 10:32:18: A. Yes, I was the project manager on-site.
Line 3, 10:32:18: Q. Can you describe what happened on that day?
Line 4, 10:32:18: A. There was an accident involving a pedestrian and a company vehicle.
--- End of Page 9 ---

--- Beginning of Page 10 ---
Line 1, 10:32:18: Q. What were the circumstances leading up to the accident?
Line 2, 10:32:18: A. The area was poorly lit, and the pedestrian unexpectedly crossed in front of the vehicle.
--- End of Page 10 ---

--- Beginning of Page 11 ---
Line 1, 10:32:18: Q. What measures were taken to secure the area?
Line 2, 10:32:18: A. We had standard safety protocols in place, but the lighting issue had not been addressed.
--- End of Page 11 ---

--- Beginning of Page 12 ---
Line 1, 10:32:18: Q. Were there any prior complaints about the lighting in that area?
Line 2, 10:32:18: A. Yes, there had been several complaints which were documented in our project logs.
--- End of Page 12 ---

--- Beginning of Page 13 ---
Line 1, 10:32:18: Q. To your knowledge, were these issues addressed before the accident?
Line 2, 10:32:18: A. They were scheduled for review, but no changes had been made by the date of the accident.
--- End of Page 13 ---

--- Beginning of Page 14 ---
Line 1, 10:32:18: CROSS-EXAMINATION
Line 2, 10:32:18: BY MS. STONE:
--- End of Page 14 ---

--- Beginning of Page 15 ---
Line 1, 10:32:18: Q. Ms. Doe, isn't it true that despite the complaints, no urgent action was taken by your company?
Line 2, 10:32:18: A. That is correct, but we followed the procedures as dictated by our safety guidelines.
--- End of Page 15 ---

--- Beginning of Page 16 ---
Line 1, 10:32:18: Q. Are you saying that the safety of pedestrians was not a priority for your project?
Line 2, 10:32:18: A. Safety is always our priority, but unfortunately, the lighting issue was not resolved in time.
--- End of Page 16 ---

--- Beginning of Page 17 ---
Line 1, 10:32:18: Page 2
Line 2, 10:32:18: EXHIBITS
Line 3, 10:32:18: Exhibit 1 - Project Safety Guidelines
Line 4, 10:32:18: Exhibit 2 - Complaint Log Entries
--- End of Page 17 ---
--- Beginning of Page 18 ---
Line 1, 10:32:18: (Whereupon, the deposition was concluded at 11:58:18.)
--- End of Page 18 

"""


In [ ]:
get_summary(query, model, tokenizer)

2024-05-28 15:29:46.786024: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 15:29:46.786089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 15:29:46.793015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
torch.cuda.is_available()

True

In [8]:
print("Is CUDA available?", torch.cuda.is_available())
print("Device:", DEVICE)

Is CUDA available? True
Device: cuda:0


In [10]:

#Function to generate a summarization prompt for a given deposition, now including the corresponding summary
# Determine the device to run the model on: GPU if available, otherwise CPU
DEVICE = 'cuda:0'
# Define a function to generate a summary of contradictions from a deposition transcript
def get_summary(query: str, model, tokenizer) -> str:
    # Define a template for the prompt that will be given to the model
    prompt_template = """
    ## Instructions
    Create a 80 word long summary of the deposition
    The deposition summary should concisely include the deponent’s name,
    case details (case name and number,court),date, and location of the deposition.
    It should Mention what the case was about generally .It should outline key topics discussed,
    summarize pivotal statements, and note significant objections and rulings.
    Additionally, mention crucial discrepancies or admissions relevant to the case’s core issues.
    The summary should also list exhibits referenced during the deposition,
    and note the preparer’s information and date of summary completion.
    
    ## Deposition
    {query}
    
    ## Summary
    """

    # Insert the actual deposition transcript into the prompt template
    prompt = prompt_template.format(query=query)
    # Tokenize the prompt, converting it to a format suitable for the model
    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    # Move the tokenized inputs to the appropriate device (GPU or CPU)
    model_inputs = encodeds.to(DEVICE)
    # Generate a response from the model
    generated_ids = model.generate(
        **model_inputs,                # Pass the model inputs
        max_new_tokens=1000,           # Limit the output to 1000 new tokens
        do_sample=False,                # Enable sampling to generate diverse outputs
        pad_token_id=tokenizer.eos_token_id  # Use the end-of-sequence token for padding
    )
    # Decode the generated token IDs back into human-readable text
    decoded = tokenizer.batch_decode(generated_ids)
    generated_text = decoded[0]
    
    return generated_text



In [9]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0
